In [1551]:
import pandas as pd
from collections import defaultdict
from itertools import combinations
import copy
import random


In [1552]:
df = pd.read_csv('form.csv')

In [1553]:
class Driver:
    def __init__(self, name, amount_seats, pickup_location, service_type, plans):
        self.name = name
        self.amount_seats = amount_seats
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name


class Rider:
    def __init__(self, name, pickup_location, service_type, plans):
        self.name = name
        self.pickup_location = pickup_location
        self.service_type = service_type
        self.plans = plans

    def __hash__(self):
        return hash(self.name)

    def __eq__(self, other):
        return isinstance(other, Rider) and self.name == other.name

In [1554]:
# Maximum number of distinct locations a driver shoud drive to pick up their passengers
MAXIMUM_LOCATION_THRESHOLD = 1

# Default number of passengers the driver's car can hold.
PASSENGER_LIMIT = 4


In [1555]:
people_list = df["Name"]
locations_list = df["Where would you like to be picked up?"]
service_list = df["Which service are you attending?"]
is_driver_list = df["Are you a driver?"]
plans_list = df["Preferred after church plans?"]

drivers = set()
riders = set()

for i in range(len(people_list)):
    if is_driver_list[i] == "Yes!":
        drivers.add(Driver(people_list[i], PASSENGER_LIMIT, locations_list[i], service_list[i], plans_list[i]))
    else:
        riders.add(Rider(people_list[i], locations_list[i], service_list[i], plans_list[i]))

print("List of all drivers:\n")
for d in drivers:
    print(d.name)

List of all drivers:

david kim
Clay Murphy
Olivia Chang
Ric Chang
Oriana Tang
Rebecca Lu
AZ Ellis
Jack Havemann
Jocelyn Lee
Jenny Sohn
Josh Paik
Lia kim
Matthew Ahn
Jonathan Mak


In [1556]:
def print_assignment(assignments): # driver to list of rider object
    for driver, riders in assignments.items():
        print(f"\nDriver: {driver.name} {driver.pickup_location.split()[0]}")
        for rider in riders:
            print(rider.name, rider.pickup_location.split()[0])

In [1557]:
'''
Change amount of seats from the default number of 4 to another amount.
'''
for driver in drivers:
    if driver.name == "Ellen Kang":
        driver.amount_seats = 6
    if driver.name == "Oriana Tang":
        driver.amount_seats = 3

In [1558]:
'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders = {
    "Rebecca Lu": {"Seojin Kwon", "Khang Le"},
    "Oriana Tang": {"Sam Ko"},
    "Josh Paik": {"Jane Yoo"},
    "Olivia Chang":  {"Melody Hong", "Christina Ko"},
    "Oliviaff Chang":  {"Melodffy Hong", "Christina Ko"}

}

'''
Assign riders who should ride together
'''
rider_groups = [
    {"Austin Lee", "Jane Yoo"},
    {"Lucy Han", "Joanna Wei"},
    {"Shayla Nguyen", "Pedro Flores-Teran"}
   
]



In [1559]:
def assign_riders_greedy(drivers, riders):
    print(len(riders))

    drivers = copy.deepcopy(drivers)
    riders = copy.deepcopy(riders)

    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.service_type].append(rider)
    
    driver_name_to_object = dict()
    for driver in drivers:
        driver_name_to_object[driver.name] = driver

    rider_name_to_object = dict()
    for rider in riders:
        rider_name_to_object[rider.name] = rider

    # Assign passengers who need to ride with a specific driver
    for driver_name, required_rider_names in driver_required_riders.items():
        
        # Check if the driver actually signed up
        if driver_name not in driver_name_to_object:
            continue

        time = driver_name_to_object[driver_name].service_type
        possible_riders = riders_by_time[time]
        for rider_name in required_rider_names:

            # Check if the passenger actually signed up
            if rider_name not in rider_name_to_object:
                continue
            rider = rider_name_to_object[rider_name]
            driver = driver_name_to_object[driver_name]
            if rider in unassigned_riders:
                assignments[driver].append(rider)
                unassigned_riders.remove(rider)
                driver.amount_seats -= 1
        # print_assignment(assignments)

    
    # Assign groups of riders.
    # List of list of riders objects
    rider_groups_obs = []
    for group in rider_groups:
        group_obs = [rider_name_to_object[rider_name] for rider_name in group]
        rider_groups_obs.append(group_obs)

    for group in rider_groups_obs:
        if all(r in unassigned_riders for r in group):
            for driver in drivers:
                if driver.amount_seats < len(group):
                    continue

                combined_group = assignments[driver] + group
                total_locations = {r.pickup_location for r in combined_group}
                if len(total_locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    driver.amount_seats -= len(group)
                    break


    
    # Greedily assign passengers to drivers.
    for driver in drivers:
        # print(f'\nDriver {driver.name}')
       

        time = driver.service_type
        possible_riders = riders_by_time[time]

        shuffled_riders = list(possible_riders)
        random.shuffle(shuffled_riders)

        found_set = False
        # Try forming groups of the amount that the passenger can hold all the way down to 0 passengers.
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(shuffled_riders, group_size):
                combined_group = list(group) + assignments[driver]
                locations = {r.pickup_location for r in combined_group} 
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD and all(r in unassigned_riders for r in group):
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True
                    break  # Move to next driver
    print(f'unassigned riders: {[rider.name for rider in unassigned_riders]}')
    return (assignments, unassigned_riders)
rides_to_assignments, unassigned_riders_to = assign_riders_greedy(drivers, riders)

46
unassigned riders: []


In [1560]:
print_assignment(rides_to_assignments)



Driver: Rebecca Lu Off
Khang Le Off
Seojin Kwon Off

Driver: Oriana Tang Off
Sam Ko Off
김예림 Off
April Tong Off

Driver: Josh Paik Off
Jane Yoo Off

Driver: Olivia Chang Life
Christina Ko Life
Melody Hong Life
Nathanael Wang Life
Grace Park Life

Driver: david kim Off
Pedro Flores-Teran Off
Shayla Nguyen Off
Faith Chen Off
Austin Lee Off

Driver: Clay Murphy Life
Elie Park South
Hannah Zhang South
Aaron duong South
Jiwang Lee South

Driver: Ric Chang Off
Joann Jung South
Israel Haile South
Taeho Choe South
Daniel Kuo South

Driver: AZ Ellis South
Sehyun Jung Life
Ethan Yu Life
Joanna Wei Life
JJ Lee Life

Driver: Jocelyn Lee Off
Daniel Song North
Lucy Han North
Jocelyn Youn North
Darius Ajebon  North

Driver: Jenny Sohn North
Emily Yang North
Daniel Kim  North
Gabriel Ni North
helena song North

Driver: Lia kim Off
Grace Sowon Park  South
Grace Kwon South
Benjamin Kim South
Maya Habraken  South

Driver: Jack Havemann Life
Hannah Kim South
derek liang  South
Hyeongjun Son South
Kyle Hwa

In [1561]:
'''
Assign drivers who need certain riders as passengers
'''
driver_required_riders = {
    "Jonathan Mak": {"Grace Kwon", "Seojin Kwon"},
    "AZ Ellis": {"Pedro Flores-Teran"}
}

'''
Assign riders who should ride together
'''
rider_groups = [
]



In [1562]:
def assign_riders_greedy_back(drivers, riders):
    print(len(riders))

    drivers = copy.deepcopy(drivers)
    riders = copy.deepcopy(riders)

    assignments = defaultdict(list)
    unassigned_riders = set(list(riders))

    # Group drivers and riders by pickup time
    riders_by_time = defaultdict(list)
    for rider in riders:
        riders_by_time[rider.plans].append(rider)
    
    driver_name_to_object = dict()
    for driver in drivers:
        driver_name_to_object[driver.name] = driver

    rider_name_to_object = dict()
    for rider in riders:
        rider_name_to_object[rider.name] = rider

    # Assign passengers who need to ride with a specific driver
    for driver_name, required_rider_names in driver_required_riders.items():
        # Check if the driver actually signed up
        if driver_name not in driver_name_to_object:
            continue
        time = driver_name_to_object[driver_name].service_type
        possible_riders = riders_by_time[time]
        for rider_name in required_rider_names:

            # Check if the passenger actually signed up
            if rider_name not in rider_name_to_object:
                continue
            driver = driver_name_to_object[driver_name]
            if rider in unassigned_riders:
                assignments[driver].append(rider)
                unassigned_riders.remove(rider)
                driver.amount_seats -= 1
        # print_assignment(assignments)

    
    # Assign groups of riders.
    # List of list of riders objects
    rider_groups_obs = []
    for group in rider_groups:
        group_obs = [rider_name_to_object[rider_name] for rider_name in group]
        rider_groups_obs.append(group_obs)

    for group in rider_groups_obs:
        if all(r in unassigned_riders for r in group):
            for driver in drivers:
                if driver.amount_seats < len(group):
                    continue

                combined_group = assignments[driver] + group
                total_locations = {r.pickup_location for r in combined_group}
                if len(total_locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    driver.amount_seats -= len(group)
                    break
    
    # Greedily assign passengers to drivers.
    for driver in drivers:
        # print(f'\nDriver {driver.name}')
       
        time = driver.plans
        possible_riders = riders_by_time[time]
        shuffled_riders = list(possible_riders)
        random.shuffle(shuffled_riders)

        found_set = False
        # Try forming groups of the amount that the passenger can hold all the way down to 0 passengers.
        for group_size in range(driver.amount_seats, 0, -1):
            if found_set:
                break
            for group in combinations(shuffled_riders, group_size):
                combined_group = list(group) + assignments[driver]
                locations = {r.pickup_location for r in combined_group} 
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD and all(r in unassigned_riders for r in group):
                    assignments[driver].extend(group)
                    for r in group:
                        unassigned_riders.remove(r)
                    found_set = True
                    break  # Move to next driver

    print(f'unassigned riders before using flexible: {[rider.name for rider in unassigned_riders]}')

    # Assign drivers who are flexible here if there are unassigned riders
    plan_groups = defaultdict(list)
    for rider in unassigned_riders:
        plan_groups[rider.plans].append(rider)
    
    for driver in drivers:
        if "Flexible" not in driver.plans:
            continue
        if driver.amount_seats <= 0:
            continue

        # Determine the plan of already assigned riders (if any)
        assigned_riders = assignments[driver]
        existing_plan = None
        if assigned_riders:
            existing_plan = assigned_riders[0].plans
            # Validate all already assigned riders have the same plan

        print("Flexible Driver", driver.name, driver.plans)
        for plan, riders_plans in plan_groups.items():
            if (existing_plan is not None and "Flexible" not in plan and \
                "Back home" not in plan and "Lunch" not in plan and plan != existing_plan):
                continue  # skip if not matching the existing plan

            available_riders = [r for r in riders_plans if r in unassigned_riders]
            if not available_riders:
                continue

            seats_remaining = driver.amount_seats
            print("ddd", driver.name, seats_remaining, len(assignments[driver]))
            if seats_remaining <= 0:
                continue

            # group_to_assign = available_riders[:seats_remaining]
            remaining_seats = driver.amount_seats - len(assignments[driver])
            group_to_assign = available_riders[:remaining_seats]

            if not group_to_assign:
                continue

            # Assign
            assignments[driver].extend(group_to_assign)
            for rider in group_to_assign:
                print("Removing rider", rider.name)
                unassigned_riders.remove(rider)
                plan_groups[plan].remove(rider)

            driver.amount_seats -= len(group_to_assign)
            print(f"[DEBUG] {driver.name} got {len(group_to_assign)} riders of plan '{plan}' — now has {len(assignments[driver])} total")
            break  # move to next flexible driver
        print(f'unassigned riders after using flexible: {[rider.name for rider in unassigned_riders]}')

    # Final attempt: assign unassigned FLEXIBLE riders to any "Back home" or "Lunch" cars if space allows
    flexible_unassigned = [r for r in list(unassigned_riders) if "Flexible" in r.plans]

    for rider in flexible_unassigned:
        for driver in drivers:
            if driver.amount_seats <= 0:
                continue
            if "Back home" in driver.plans or "Lunch" in driver.plans:
                current_riders = assignments[driver]
                if len(current_riders) >= driver.amount_seats:
                    continue  # skip if already full just in case amount_seats got out of sync

                combined_group = current_riders + [rider]
                locations = {r.pickup_location for r in combined_group}
                if len(locations) <= MAXIMUM_LOCATION_THRESHOLD:
                    assignments[driver].append(rider)
                    unassigned_riders.remove(rider)
                    driver.amount_seats -= 1
                    print(f"[FLEXIBLE FILL] Assigned {rider.name} to {driver.name} ({driver.plans})")
                    break  # Stop looking for cars for this rider

    print(f'unassigned riders after using flexible: {[rider.name for rider in unassigned_riders]}')

    return (assignments, unassigned_riders)

rides_back_assignments, unassigned_riders_back = assign_riders_greedy_back(drivers, riders)

46
unassigned riders before using flexible: ['Seojin Kwon', 'Daniel Kim ', 'Faith Chen', 'Joanna Wei', 'Jane Yoo', 'Jeffery Huang', '김예림', 'Austin Lee']
Flexible Driver david kim Flexible 💚
ddd david kim 4 4
ddd david kim 4 4
unassigned riders after using flexible: ['Seojin Kwon', 'Daniel Kim ', 'Faith Chen', 'Joanna Wei', 'Jane Yoo', 'Jeffery Huang', '김예림', 'Austin Lee']
Flexible Driver Clay Murphy Flexible 💚
ddd Clay Murphy 4 4
ddd Clay Murphy 4 4
unassigned riders after using flexible: ['Seojin Kwon', 'Daniel Kim ', 'Faith Chen', 'Joanna Wei', 'Jane Yoo', 'Jeffery Huang', '김예림', 'Austin Lee']
Flexible Driver Ric Chang Flexible 💚
ddd Ric Chang 4 2
Removing rider Seojin Kwon
[DEBUG] Ric Chang got 1 riders of plan 'Flexible 💚' — now has 3 total
unassigned riders after using flexible: ['Daniel Kim ', 'Faith Chen', 'Joanna Wei', 'Jane Yoo', 'Jeffery Huang', '김예림', 'Austin Lee']
unassigned riders after using flexible: ['Daniel Kim ', 'Faith Chen', 'Joanna Wei', 'Jane Yoo', 'Jeffery Huang'

In [1563]:
print_assignment(rides_back_assignments)


Driver: Jonathan Mak North
Elie Park South

Driver: david kim Off
Ella Lu South
Hyeongjun Son South
Joann Jung South
Ella South

Driver: Clay Murphy Life
April Tong Off
Sam Ko Off
Pedro Flores-Teran Off
Shayla Nguyen Off

Driver: Olivia Chang Life
Jiwang Lee South
Hannah Kim South
Grace Sowon Park  South
derek liang  South

Driver: Ric Chang Off
Gabriel Ni North
Jocelyn Youn North
Seojin Kwon Off

Driver: Oriana Tang Off
Melody Hong Life

Driver: Rebecca Lu Off
Grace Park Life
Sehyun Jung Life
Christina Ko Life
Nathanael Wang Life

Driver: AZ Ellis South
Maya Habraken  South
Hannah Zhang South
Benjamin Kim South
Grace Kwon South

Driver: Jocelyn Lee Off
Rachel Kim South
Daniel Kuo South
Kyle Hwang South
Israel Haile South

Driver: Jenny Sohn North
Daniel Song North
Emily Yang North
Lucy Han North
Darius Ajebon  North

Driver: Lia kim Off
JJ Lee Life
Ethan Yu Life

Driver: Josh Paik Off
Khang Le Off

Driver: Jack Havemann Life
helena song North

Driver: Matthew Ahn Life
Taeho Choe Sout

In [1564]:
from openpyxl import Workbook
from openpyxl.styles import Font, Alignment, PatternFill
from openpyxl.utils import get_column_letter

# # Create a new workbook and select the active sheet
# wb = Workbook()
# ws = wb.active
# ws.title = "My Sheet"

# # Write text into specific cells
# ws["A1"] = "Name"
# ws["B1"] = "Score"
# ws["A2"] = "Alice"
# ws["B2"] = 95
# ws["A3"] = "Bob"
# ws["B3"] = 88

# # Apply bold font to headers
# bold_font = Font(bold=True)
# ws["A1"].font = bold_font
# ws["B1"].font = bold_font

# # Center align header text
# center_alignment = Alignment(horizontal="center")
# ws["A1"].alignment = center_alignment
# ws["B1"].alignment = center_alignment

# # Set background fill color for header
# header_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
# ws["A1"].fill = header_fill
# ws["B1"].fill = header_fill

# # Save to file
# # wb.save("formatted_output.xlsx")

In [1565]:
location_colors = {
    "North": "FFd9ead3",     # light green 3
    "South": "FF93CCEA",     # Light Cornflower Blue 3 
    "Off": "FFFFFFED",       # light yellow
    "Life": "fff4cccc",      # green
    "Back home 💙": "FFD9D2E9",      # light purple
    "RJM": "FFEAD1DC",       # light pink
    "Lunch 💛": "FFFCE5CD",     # light orange
    "Flexible 💚": "FFCFE2F3",        # light blue
    "Refreshments": "FFB6D7A8",
    "NLK 🧡": "FFD9D9D9",

}

In [1566]:
def export_assignments_to_excel(rides_to, unassigned_riders_to, rides_from, unassigned_riders_from, output_filename="ride_assignments.xlsx"):
    wb = Workbook()
    ws = wb.active
    ws.title = "Ride Assignments"

    def place_assignments(assignments, unassigned_riders, start_col, key_col, sort_key_driver, sort_key_rider,
                          driver_color_key, rider_color_key, label_plan=False, include_rider_keys_in_legend=True):
        col = start_col
        row = 2
        curr_car_count = 1
        max_passengers = 0
        local_used_cols = set()
        used_keys = set()

        sorted_assignments = sorted(assignments.items(), key=lambda item: sort_key_driver(item[0]))

        for driver, riders in sorted_assignments:
            driver_text = f"Driver: {driver.name}"
            driver_cell = ws.cell(row=row, column=col, value=driver_text)

            driver_key = driver_color_key(driver)
            used_keys.add(driver_key)
            fill_color = location_colors.get(driver_key, "FFFFFF")
            driver_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            max_passengers = max(max_passengers, len(riders))

            sorted_riders = sorted(riders, key=sort_key_rider)
            for i, rider in enumerate(sorted_riders):
                rider_text = f"{rider.name}"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider)
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

            if curr_car_count % 5 == 0:
                col = start_col
                row += min(8, max_passengers + 3)
            else:
                col += 1
            curr_car_count += 1

        if unassigned_riders:
            col += 1
            driver_cell = ws.cell(row=row, column=col, value="UNASSIGNED DRIVERS")
            driver_cell.fill = PatternFill(start_color="FFCCCCCC", end_color="FFCCCCCC", fill_type="solid")
            driver_cell.alignment = Alignment(horizontal="center")
            driver_cell.font = Font(bold=True, underline="single")
            local_used_cols.add(col)

            sorted_unassigned = sorted(unassigned_riders, key=sort_key_rider)
            for i, rider in enumerate(sorted_unassigned):
                rider_text = f"{rider.name} ({rider.plans})" if label_plan else f"{rider.name}"
                rider_cell = ws.cell(row=row + 1 + i, column=col, value=rider_text)
                rider_key = rider_color_key(rider)
                if include_rider_keys_in_legend:
                    used_keys.add(rider_key)
                fill_color = location_colors.get(rider_key, "FFFFFF")
                rider_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
                local_used_cols.add(col)

        # Create key
        key_row = 2
        for key in sorted(used_keys):
            key_cell = ws.cell(row=key_row, column=key_col, value=key)
            fill_color = location_colors.get(key, "FFFFFF")
            key_cell.fill = PatternFill(start_color=fill_color, end_color=fill_color, fill_type="solid")
            key_row += 1

        return local_used_cols

    # Define sort and color strategies
    sort_by_pickup = lambda obj: obj.pickup_location.split()[0]
    sort_by_plans = lambda obj: obj.plans
    color_by_pickup = lambda obj: obj.pickup_location.split()[0]
    color_by_plans = lambda obj: obj.plans

    # rides_to (left): include rider pickup locations in key
    used_to = place_assignments(
        rides_to,
        unassigned_riders_to,
        start_col=3,
        key_col=2,
        sort_key_driver=sort_by_pickup,
        sort_key_rider=sort_by_pickup,
        driver_color_key=color_by_pickup,
        rider_color_key=color_by_pickup,
        label_plan=False,
        include_rider_keys_in_legend=True
    )

    # rides_from (right): do NOT include rider pickup in key
    used_from = place_assignments(
        rides_from,
        unassigned_riders_from,
        start_col=11,
        key_col=10,
        sort_key_driver=sort_by_plans,
        sort_key_rider=sort_by_plans,
        driver_color_key=color_by_plans,
        rider_color_key=color_by_pickup,
        label_plan=True,
        include_rider_keys_in_legend=False
    )

    for col_index in used_from.union(used_to).union({2, 10}):
        col_letter = get_column_letter(col_index)
        max_length = 0
        for row_cells in ws.iter_rows(min_col=col_index, max_col=col_index):
            for cell in row_cells:
                if cell.value:
                    max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[col_letter].width = max_length + 1

    wb.save(output_filename)
    print(f"Excel file saved to {output_filename}")

In [1567]:
export_assignments_to_excel(
    rides_to=rides_to_assignments,
    unassigned_riders_to=unassigned_riders_to,
    rides_from=rides_back_assignments,
    unassigned_riders_from=unassigned_riders_back
)

Excel file saved to ride_assignments.xlsx
